In [17]:
# Make sure to run Input_data.ipynb first

# Load all input data
%store -r 

In [18]:
# Make sure to run Input_data.ipynb first

# Load all input data
%store -r 

In [19]:
# Make sure to run Input_data.ipynb first

# Load all input data
%store -r 

In [20]:
# Make sure to run Input_data.ipynb first

# Load all input data
%store -r 

In [21]:
import pyomo.environ as pe
import pyomo.opt as po

In [22]:
# Initialize Model
model = pe.ConcreteModel()

# Sets
model.F = pe.Set(initialize = factories ,ordered = False)
model.T = pe.Set(initialize = transshipment_points ,ordered = False)
model.C = pe.Set(initialize = customers ,ordered = False)
model.M = pe.Set(initialize = modes ,ordered = False)
model.P = pe.Set(initialize = periods, ordered = False)
model.W = pe.Set(initialize = warehouses, ordered = False)

# Decision Variables
# NOTE je kan ook lower en upper bounds geven aan de waarden
model.x = pe.Var(model.F, model.C, model.P, domain = pe.NonNegativeReals)                   # direct tonnes                         !!!TODO : meer uitleg !!!
model.y = pe.Var(model.T, model.C , model.P, domain = pe.NonNegativeReals)                  # indirect tonnes                       !!!TODO : meer uitleg !!!
model.q_bl = pe.Var(model.F, block_trans, model.P, domain = pe.NonNegativeReals)            # quantity block mode                   !!!TODO : meer uitleg !!!  
model.q_s_w = pe.Var(model.F, single_trans, model.P, domain = pe.NonNegativeReals)          # quantity single wagon mode            !!!TODO : meer uitleg !!!
model.q_ss = pe.Var(model.F, shortsea_trans, model.P, domain = pe.NonNegativeReals)         # quantity shortsea ship mode           !!!TODO : meer uitleg !!!
model.q_ba = pe.Var(model.F, barge_trans, model.P, domain = pe.NonNegativeReals)            # quantity barge mode                   !!!TODO : meer uitleg !!!
model.td = pe.Var(model.F, model.C, model.P, domain = pe.NonNegativeIntegers)               # number of trucks via direct route     !!!TODO : meer uitleg !!!
model.tid = pe.Var(model.T, model.C, model.P, domain = pe.NonNegativeIntegers)              # number of trucks via indirect route   !!!TODO : meer uitleg !!!

model.inv_level = pe.Var(model.W, model.P, domain = pe.NonNegativeReals)

#Additional decision variables 
model.barge_dis = pe.Var(model.F, barge_trans, model.P, domain=pe.Binary)

In [23]:
objExpr = sum(dist_fact_cust[i, j] * transfer_cost * model.x[i,j,p] for i in model.F for j in model.C for p in model.P)     \
        + sum(block_train_cost[i, j] * model.q_bl[i,j,p] for i in model.F for j in block_trans for p in model.P)            \
        + sum(single_wagon_cost[i, j] * model.q_s_w[i,j,p] for i in model.F for j in single_trans for p in model.P)         \
        + sum(shortsea_cost[i, j] * model.q_ss[i,j,p] for i in model.F for j in shortsea_trans for p in model.P)            \
        + sum(barge_cost[i, j] * model.q_ba[i,j,p] * (1-discount*model.barge_dis[i,j,p])for i in model.F for j in barge_trans for p in model.P)                  \
        + sum(dist_point_cust[i, j] * transfer_cost * model.y[i,j,p] for i in model.T for j in model.C for p in model.P)    \
        + sum(handling_costs[i] * model.y[i,j,p] for i in model.T for j in model.C for p in model.P)                        \
        + sum(model.td[i,j,p] * truck_fixed_cost for i in model.F for j in model.C for p in model.P)                        \
        + sum(model.tid[i,j,p] * truck_fixed_cost for i in model.T for j in model.C for p in model.P)                       \
       
model.obj = pe.Objective(expr = objExpr, sense = pe.minimize)

In [24]:
model.trucksDirect = pe.ConstraintList()
for p in model.P:
    for i in model.F:
        for j in model.C:
            expression = model.td[i,j,p] >= model.x[i,j,p] / truck_cap
            model.trucksDirect.add(expression)

model.trucksIndirect = pe.ConstraintList()
for p in model.P:
    for i in model.T:
        for j in model.C:
            expression = model.tid[i,j,p] >= model.y[i,j,p] / truck_cap
            model.trucksIndirect.add(expression)


model.prodCap = pe.ConstraintList()
for p in model.P:
    for i in model.F:
        expression = sum(model.x[i,j,p] for j in model.C) + sum(model.q_bl[i,j,p] for j in block_trans) + sum(model.q_s_w[i,j,p] for j in single_trans) + sum(model.q_ss[i,j,p] for j in shortsea_trans) + sum(model.q_ba[i,j,p] for j in barge_trans) <= production_cap[i]
        model.prodCap.add(expression)

model.meetDemand = pe.ConstraintList()
for p in model.P:
    for j in model.C:
        expression = sum(model.x[i,j,p] for i in model.F) + sum(model.y[i,j,p] for i in model.T) >= demands[p][j] - model.inv_level[j,p]
        model.meetDemand.add(expression)


model.flowBalance = pe.ConstraintList()
for p in model.P:
    total_quantity_trans = {"Robert Schmidtz": 0, "Wiechers": 0, "Haeger und Schmidt": 0, "Neska/UCT Neuss": 0,
			"WTA": 0, "Siefert spedition": 0, "Rhenus logistics": 0}
    for j in block_trans:
        expression_bl = sum(model.q_bl[i,j,p] for i in model.F)
        expression_s_w = sum(model.q_s_w[i,j,p] for i in model.F) 
        total_train = expression_bl + expression_s_w
        total_quantity_trans[j] += total_train
    for j in shortsea_trans:
        expression_ss = sum(model.q_ss[i,j,p] for i in model.F) 
        total_quantity_trans[j] += expression_ss
    for j in barge_trans:
        expression_ba = sum(model.q_ba[i,j,p] for i in model.F) 
        total_quantity_trans[j] += expression_ba
    for j in model.T: 
        expression = total_quantity_trans[j] == sum(model.y[j,k,p] for k in model.C)
        model.flowBalance.add(expression)

model.inventory = pe.ConstraintList()
for w in model.W:
    expression = model.inv_level[w,1] == InvLev[w] + sum(model.x[i,w,1] for i in model.F) + sum(model.y[i,w,1] for i in model.T) - demands[1][w] 
    model.inventory.add(expression)  

model.update_inventory = pe.ConstraintList()
for p in model.P:
    for w in model.W: 
        if p > 1:
            expression = model.inv_level[w,p] == model.inv_level[w,p-1] + sum(model.x[i,w,p] for i in model.F) + sum(model.y[i,w,p] for i in model.T) - demands[p][w]
            model.update_inventory.add(expression)

model.inventory_cap = pe.ConstraintList()
for p in model.P:
    for w in model.W:
        expression = model.inv_level[w,p] <= InvCap[w]
        model.inventory_cap.add(expression)

model.barge_discount = pe.ConstraintList()
for p in model.P:
    for i in model.F:
        for j in barge_trans:
            expression = model.barge_dis[i,j,p] <= model.q_ba[i,j,p] / 100
            model.barge_discount.add(expression)

In [25]:
solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee = True, timelimit=360) # timelimit werkt wel maar vgm blijft ie op de achtergrond runnen


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-03-21
Using license file /Users/tessavanliempt/gurobi.lic
Read LP format model from file /var/folders/qp/3z7wqsnn5xn47x6dpmtpq_vh0000gp/T/tmp85p0cv96.pyomo.lp
Reading time = 0.28 seconds
x2011: 1201 rows, 2011 columns, 4973 nonzeros
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1201 rows, 2011 columns and 4973 nonzeros
Model fingerprint: 0x0fa90ed8
Model has 60 quadratic objective terms
Variable types: 1151 continuous, 860 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [2e+00, 1e+05]
  QObjective range [5e-01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+03]
Found heuristic solution: objective 1.330880e+14
Presolve removed 109 rows and 81 columns
Presolve

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/Users/tessavanliempt/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/tessavanliempt/opt/anaconda3/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/tessavanliempt/opt/anaconda3/lib/python3.9/site-packages/pyomo/common/tee.py", line 505, in _mergedReader
    ready_handles = select(
OSError: [Errno 9] Bad file descriptor


TimeoutExpired: Command '['/usr/local/bin/gurobi.sh']' timed out after 363.59476036500007 seconds

In [26]:
print("Objective value = "+ str(pe.value(model.obj)))

ERROR: evaluating object as numeric value: x[South Wales,Boenen,1]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object x[South Wales,Boenen,1]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.ScalarObjective'>)
    No value for uninitialized NumericValue object x[South Wales,Boenen,1]


ValueError: No value for uninitialized NumericValue object x[South Wales,Boenen,1]